In [1]:
import torch
from torch import utils
from torchvision import datasets, transforms

from torch.utils.data import DataLoader


import matplotlib
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
path='./MNIST_data'

#### Смотрим на данные

In [3]:
train_data = datasets.MNIST(path, train=True, download=True)
train_data


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./MNIST_data
    Split: Train

In [4]:
# параметры датасета
len_train = len(train_data)
print(f"кол-во элементов в датасете: {len_train}")
print(f"тип элемента:  {type(train_data[0])}")
print(f"длинна элемента:  {len(train_data[0])}")

кол-во элементов в датасете: 60000
тип элемента:  <class 'tuple'>
длинна элемента:  2


In [5]:
# посмотрим на содержимое 
print(type(train_data.data[0]))
print(train_data.data[0].shape)



<class 'torch.Tensor'>
torch.Size([28, 28])


#### Вычисления и преобразования

In [36]:
def get_train_mean_std(train_set):
    """
    Получаем тренировочный датасет
    Возвращаем кортеж (mean, std) 
    ==============================
    """   
    #tr = transforms.ToTensor()
    #items = tr(train_set.data.numpy())
    
    # обращаемся к сырым дынным и трансформируем их 
    m_ = train_set.data.numpy().mean()/255.0
    std_ = train_set.data.numpy().std()/255.0
    return m_, std_

In [37]:
# получим статистики
k_mean, k_std = get_train_mean_std(train_data)
print(f"Mean = {k_mean}\nStd = {k_std}")

Mean = 0.1306604762738429
Std = 0.30810780385646264


#### проверяем результат

In [38]:
# создаем свой трансформ
mnist_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((k_mean,), (k_std,)),
                ])

In [39]:
# подготавливаем данные и нормализуем
items = mnist_transform(train_data.data.numpy())

print(items.shape)
print()
print(f"Mean = {items.mean().round()}\nStd = {items.std()}")


torch.Size([28, 60000, 28])

Mean = 0.0
Std = 1.0000001192092896


### OK!


##### Проверяем transform - вариант 1

In [40]:
train_data_mod = datasets.MNIST(path, train=True, download=True, transform=mnist_transform)

проверка

In [41]:
mnist_dl = DataLoader(dataset=train_data_mod, batch_size=len(train_data_mod))
items, _ = next(iter(mnist_dl))

print(items.shape)
print(f"Mean = {items.mean().round()}\nStd = {items.std().round()}")

torch.Size([60000, 1, 28, 28])
Mean = -0.0
Std = 1.0


##### Создаем свой класс transform - вариант 2
тоже самое но скрываем все в классе

In [42]:
import torchvision.transforms.functional as TF

class MNIST_Transform:
    
    def __init__(self, mean, std, inplace=False):
        self.mean = mean
        self.std = std
        self.inplace = inplace

    def __call__(self, image):
        """
        Args:
            image: PIL image of size (C, H, W) to be normalized.

        Returns:
            Tensor: Normalized Tensor image.
        """
        return TF.normalize(TF.to_tensor(image), self.mean, self.std, self.inplace)


    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

    

In [43]:
mnist_class_transform = MNIST_Transform((k_mean,), (k_std,))

In [44]:
train_data_mod = datasets.MNIST(path, train=True, download=True, transform=mnist_class_transform)

проверка

In [45]:
mnist_dl = DataLoader(dataset=train_data_mod, batch_size=len(train_data_mod))
items, _ = next(iter(mnist_dl))

print(items.shape)
print(f"Mean = {items.mean().round()}\nStd = {items.std().round()}")

torch.Size([60000, 1, 28, 28])
Mean = -0.0
Std = 1.0
